In [13]:
import tensorflow as teras

import pandas as pandinha

In [14]:
nome_das_colunas = ['LV-ActivePower-kW', 'Wind-Speed-m/s', 'Theoretical-Power-Curve-KWh', 'Wind-Direction', 'potenciado']
funcionamento = ['Fora do Limite Aceitável', 'Dentro do Limite Aceitável']

In [15]:
arquivo = pandinha.read_csv(teras.keras.utils.get_file(
    'raw_turbina_scala.csv',
    'https://raw.githubusercontent.com/vhsmdev/analytics-turbina-eolica/main/data/raw/raw_turbina_scala.csv'))

potenciado = []

for p, potencia in enumerate(list(arquivo['LV ActivePower (kW)'])):
  if potencia >= (arquivo.loc[p, 'Theoretical_Power_Curve (KWh)'] * 0.95) and potencia <= (arquivo.loc[p, 'Theoretical_Power_Curve (KWh)'] * 1.05):
    potenciado.append(1)

  else:
    potenciado.append(0)

arquivo['potenciado'] = potenciado

arquivo = pandinha.concat([arquivo, arquivo.loc[(arquivo['LV ActivePower (kW)'] == 0.0) |
                                                  (arquivo['Wind Speed (m/s)'] == 0.0) |
                                                  (arquivo['Theoretical_Power_Curve (KWh)'] == 0.0) |
                                                  (arquivo['Wind Direction (°)'] == 0.0)]])

arquivo = arquivo.drop_duplicates(keep = False).drop('Date/Time', axis = 1)

arquivo.columns = nome_das_colunas

treino = arquivo[9821:]

prova = arquivo[:9821]

treino_y = treino.pop('potenciado')

prova_y = prova.pop('potenciado')

In [16]:
def fu_entrada(recursos, rotulos, treinamento = True, tamanho_lote = 256):
  dados = teras.data.Dataset.from_tensor_slices((dict(recursos), rotulos))

  if treinamento:
    dados = dados.shuffle(1000).repeat()

  return dados.batch(tamanho_lote)

In [17]:
colunas_de_recurso = []

for chave in treino.keys():
  colunas_de_recurso.append(teras.feature_column.numeric_column(key = chave))

In [18]:
classificador = teras.keras.Sequential([
  teras.keras.layers.DenseFeatures(colunas_de_recurso),
  teras.keras.layers.Dense(30, activation = 'relu'),
  teras.keras.layers.Dense(10, activation = 'relu'),
  teras.keras.layers.Dense(2, activation = 'softmax')])

classificador.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [19]:
classificador.fit(
    fu_entrada(treino, treino_y, treinamento = True),
    steps_per_epoch = 5000)

5000/5000 [==============================] - 16s 3ms/step - loss: 0.6735 - accuracy: 0.9708


In [20]:
resultado_de_avaliacao = classificador.evaluate(
    fu_entrada(prova, prova_y, treinamento = False))

print(f'\nPrecisão do Conjunto de Provas: {resultado_de_avaliacao[-1] * 100:.2f}%')

39/39 [==============================] - 0s 2ms/step - loss: 0.1461 - accuracy: 0.9537

Precisão do Conjunto de Provas: 95.37%


In [21]:
def entrada(recursos, lote = 256):
  return teras.data.Dataset.from_tensor_slices(dict(recursos)).batch(lote)

In [22]:
recursos = ['LV-ActivePower-kW', 'Wind-Speed-m/s', 'Theoretical-Power-Curve-KWh', 'Wind-Direction']
prever = {}

In [ ]:
for recurso in recursos:
  valido = True

  while valido:
    val = input(recurso + ': ')

    if not val.isdigit(): valido = False

  prever[recurso] = [float(val)]

previsoes = classificador.predict(entrada(prever))

print(f'Previsto para {funcionamento[list(previsoes[0]).index(max(list(previsoes[0])))]}')